<h1>🚀 蒼狼 AI v2.2 - 一鍵啟動器</h1>

歡迎使用！點擊下方儲存格左側的 ▶️ **執行按鈕**來啟動您的 AI 分析平台。

**啟動流程：**
1. **環境設定**：自動下載專案程式碼並安裝所有必要的 Python 和 Node.js 套件。
2. **啟動服務**：同時啟動後端 FastAPI 和前端 Next.js 服務。
3. **建立通道**：使用 `ngrok` 建立一個安全的公開網址，以便您存取服務。
4. **嵌入介面**：**您的應用程式使用者介面將會直接顯示在此儲存格的下方**，無需離開 Colab。

---
**提示**：首次執行可能需要幾分鐘時間來下載和安裝套件，請耐心等候。

In [ ]:
#@title ▶️ 點此一鍵啟動 Wolf AI 平台
import os
import subprocess
import threading
from pyngrok import ngrok
from IPython.display import display, IFrame, HTML
import time
import requests # Added for backend health check

# --- 設定 --- 
REPO_URL = "https://github.com/hsp1234-web/wolfai_v2_jules.git"
REPO_PATH = "/content/wolfai_v2_jules"
NGROK_AUTH_TOKEN = "" # 建議填寫您的 ngrok Authtoken 以獲得更穩定連線

def stream_output(process, log_file):
    """將子進程的輸出即時寫入日誌檔案"""
    with open(log_file, 'w') as f:
        for line in iter(process.stdout.readline, ''):
            f.write(line)
            f.flush()

print("⏳ [1/5] 開始準備環境...")

if os.path.exists(REPO_PATH):
    print(f"   -> 倉庫目錄 {REPO_PATH} 已存在，正在更新...")
    subprocess.run(f'cd {REPO_PATH} && git pull', shell=True, check=True)
else:
    print(f"   -> 正在從 {REPO_URL} 下載專案...")
    subprocess.run(f'git clone {REPO_URL} {REPO_PATH}', shell=True, check=True)

print("✅ [2/5] 專案程式碼已備妥！")
print("⏳ [3/5] 正在安裝後端與前端依賴套件 (此步驟可能需要數分鐘)...")

# 安裝所有依賴
subprocess.run(f'pip install -q -r {REPO_PATH}/backend/requirements.txt', shell=True, check=True)
subprocess.run(f'npm install --prefix {REPO_PATH}/frontend', shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) # 隱藏 npm 的大量輸出

print("✅ [4/5] 所有依賴已安裝！")
print("⏳ [5/5] 正在啟動後端與前端服務...")

# 啟動後端
backend_process = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'], cwd=f'{REPO_PATH}/backend', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
threading.Thread(target=stream_output, args=(backend_process, 'backend.log')).start()
print("   -> 後端 FastAPI 服務已在背景啟動。開始健康檢查...")

    # 後端健康檢查
    backend_health_url = "http://localhost:8000/api/v1/health"
    max_retries = 30
    retry_delay = 2  # seconds
    backend_healthy = False
    for i in range(max_retries):
        print(f"   -> 健康檢查嘗試 {i+1}/{max_retries}...")
        try:
            response = requests.get(backend_health_url, timeout=5)
            if response.status_code == 200:
                print("   -> 後端健康檢查成功！服務已就緒。")
                backend_healthy = True
                break
            else:
                print(f"   -> 後端健康檢查失敗，狀態碼：{response.status_code}")
        except requests.exceptions.ConnectionError:
            print("   -> 後端健康檢查失敗，無法連線。")
        except requests.exceptions.Timeout:
            print("   -> 後端健康檢查超時。")
        except Exception as e:
            print(f"   -> 後端健康檢查發生未知錯誤: {e}")
        
        if i < max_retries - 1:
            time.sleep(retry_delay)

    if not backend_healthy:
        print("🛑 錯誤：後端服務在多次嘗試後仍未成功啟動。請檢查 backend.log 以獲取詳細資訊。")
        # 可以選擇在此處引發異常，例如：
        # raise RuntimeError("後端服務啟動失敗")
        # 但為了 Colab 的使用者體驗，我們可能僅打印錯誤並繼續，讓使用者自行判斷
    # else: # Only proceed if backend is healthy - or always proceed and let user see frontend errors
        # For now, let's always attempt to start frontend, errors will be logged.

    print("   -> 啟動前端 Next.js 服務...")
    # Start Frontend Process
    frontend_process = subprocess.Popen(
        ['npm', 'run', 'dev'], 
        cwd=f'{REPO_PATH}/frontend', 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT, 
        text=True, 
        bufsize=1,  # Line buffered
        universal_newlines=True # Ensure text mode works consistently
    )

    # Frontend Ready Check & Log Streaming
    frontend_ready_messages = [
        "ready - started server on 0.0.0.0:3000, url: http://localhost:3000",
        "- ready started server on 0.0.0.0:3000, url: http://localhost:3000",
        "event - compiled client and server successfully in",
        "ready in",
        "started server on 0.0.0.0:3000, url: http://localhost:3000" # Another common variant
    ]
    frontend_ready_messages_lower = [msg.lower() for msg in frontend_ready_messages]
    frontend_ready = False
    frontend_log_path = 'frontend.log'
    start_time = time.time()
    frontend_timeout = 180  # 3 minutes, as npm install + dev server can be slow

    print(f"   -> 等待前端服務就緒 (最多 {frontend_timeout} 秒)... 日誌同時寫入 {frontend_log_path} 和 Colab 輸出。")

    with open(frontend_log_path, 'w') as f_log:
        for line in iter(frontend_process.stdout.readline, ''):
            if not line and frontend_process.poll() is not None:
                print("   -> 前端進程已提前終止。檢查 frontend.log 以獲取詳細資訊。")
                break

            f_log.write(line)
            f_log.flush()
            # Displaying all logs in Colab can be verbose; optionally enable or only print key lines
            # print(f"   [FRONTEND LOG]: {line.strip()}", flush=True) 

            if any(ready_msg in line.lower() for ready_msg in frontend_ready_messages_lower):
                print(f"   -> 前端服務已成功偵測到就緒訊息: '{line.strip()}'")
                frontend_ready = True
                # Optional: print a few more lines from frontend to catch any immediate follow-up info
                # for _ in range(3):
                #     try:
                #         extra_line = frontend_process.stdout.readline()
                #         if extra_line:
                #             f_log.write(extra_line)
                #             f_log.flush()
                #             # print(f"   [FRONTEND LOG]: {extra_line.strip()}", flush=True)
                #         else:
                #             break
                #     except:
                #         break
                break # Exit loop once ready message found
            
            if time.time() - start_time > frontend_timeout:
                print(f"   -> 錯誤：等待前端服務就緒超時 ({frontend_timeout} 秒)。")
                # Attempt to terminate the process if it's still running
                if frontend_process.poll() is None:
                    print("   -> 嘗試終止超時的前端進程...")
                    frontend_process.terminate()
                    try:
                        frontend_process.wait(timeout=5) # Wait for termination
                        print("   -> 前端進程已終止。")
                    except subprocess.TimeoutExpired:
                        print("   -> 前端進程未能終止，可能需要手動處理。")
                        frontend_process.kill() # Force kill if terminate failed
                break

    # Final check on process status if not marked ready
    if not frontend_ready and frontend_process.poll() is not None:
        print("   -> 前端進程已結束，且未偵測到就緒訊息。請檢查 frontend.log。")

    if frontend_ready:
        print("   -> 前端 Next.js 服務已啟動並偵測到就緒狀態。")
    else:
        print("   -> 警告：前端 Next.js 服務未偵測到就緒狀態或啟動失敗。請檢查 frontend.log。ngrok 可能無法正確連線。")

# 設定 ngrok (僅在前端成功啟動後)
if frontend_ready:
    print("   -> 前端服務已就緒，正在設定 ngrok...")
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("   -> ngrok Authtoken 已設定。")

    # 建立 ngrok 通道並顯示
    try:
        print("   -> 正在建立 ngrok 通道至前端服務 (port 3000)...")
        public_url = ngrok.connect(3000).public_url
        print(f"
🎉🎉🎉 您的 AI 平台已上線！ 🎉🎉🎉")
        print(f"🌐 公開網址: {public_url}")
        display(HTML("<h2>↓ 您可以直接在下方操作 WolfAI 介面 ↓</h2>"))
        display(IFrame(public_url, width='100%', height=800))
    except Exception as e:
        print(f"
🛑 錯誤：無法建立 ngrok 通道。請檢查您的 Authtoken 或 Colab 網路狀態。錯誤訊息: {e}")
else:
    print("   -> 跳過 ngrok 設定，因為前端服務未能成功啟動。")